<a href="https://colab.research.google.com/github/wangyao049/Keras-Tutorial/blob/master/catdog_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用预训练的卷积神经网络

In [0]:
# colab连接google drive中的数据
from google.colab import drive
drive.mount('/content/gdrive')

## 1. 在你的数据集上运行卷积基，将输出保存成硬盘中的numpy数组，然后用这个数据作为输入，输入到独立的密集连接分类器中。
* 该方法不能使用数据增强
* 速度快，使用CPU就可以训练

In [0]:
# 加载在ImageNet上预训练好的vgg16模型
from keras.applications import VGG16

base_model = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150,150,3))

base_model.summary()

In [0]:
# 利用vgg16网络，对新数据集进行特征提取
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

data_dir = './gdrive/My Drive/cats_and_dogs_small'
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')
test_dir = os.path.join(data_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

# 该函数返回两个值（特征值以及标签）， 特征提取使用base_model.predict
def extract_features(dir, sample_count):
  features = np.zeros(shape=(sample_count, 4, 4, 512))
  labels = np.zeros(shape=(sample_count))
  
  generator = datagen.flow_from_directory(
        dir,
        target_size=(150,150),
        batch_size=batch_size,
        class_mode='binary')
  
  i = 0
  for inputs_batch, labels_batch in generator:
    features_batch = base_model.predict(inputs_batch)
    features[i * batch_size : (i+1) * batch_size] = features_batch
    labels[i * batch_size : (i+1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
    
    return features, labels

In [0]:
# 对所有图片进行特征提取
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

# 将这些特征展平为2D张量，然后使用Dense层进行分类 （Dense层只能处理向量数据，形如（samples，features））
train_features = np.reshape(train_features, (2000, 4*4*512))
validation_features = np.reshape(validation_features, (1000, 4*4*512))
test_features = np.reshape(test_features, (1000, 4*4*512))


In [0]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.optimizers import RMSprop

# 定义一个全连接分类器，并且在刚刚保存的数据和标签上训练这个分类器
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=4*4*512))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=RMSprop(2e-5), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_features, train_labels, batch_size=20, epochs=30, validation_data=(validation_features, validation_labels))

In [0]:
test_loss, test_acc = model.evaluate(test_features, test_labels)

In [0]:
print(test_loss)
print(test_acc)

## 2. 在顶部添加Dense层来扩展已有模型， 并在输入数据上端到端的运行整个模型。
* 该方法可以使用数据增强
* 速度慢，计算代价高，需要GPU

In [0]:
# 在卷积基上添加一个密集分类器
from keras.layers import Flatten
model2 = Sequential()
model2.add(base_model)
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

In [0]:
# 在编译和训练模型之前，一定要冻结（freeze）卷积基， 使其在训练过程中保持权重不变。
# 如果不这么做，卷积基之前学到的表示将会在训练过程中被修改，因为新添加的Dense层是随机初始化的，所以非常大的权重更新会在网络中传播，对之前学到的表示造成很大的破坏。
base_model.trainable = False

# 对训练数据进行数据增强
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# 注意，不能增强验证数据
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model2.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=2e-5),
              metrics=['acc'])

history = model2.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=50)

## 3. 微调模型（fine-tuning）
* 微调指将其顶部的几层“解冻”，并将这解冻的几层和信增加的部分联合训练。
* 只有上面的分类器已经训练好了，才能微调卷积基的顶部几层，如果分类器没有训练好，那么训练期间通过网络传播的误差信号会特别大，微调的几层之前学到的表示都会被破坏。
* 微调网络的步骤如下：
  1. 在已经训练好的基网络上添加自定义网络。
  2. 冻结基网络。
  3. 训练所添加的部分。
  4. 解冻基网络的一些层。
  5. 联合训练解冻的这些层和添加的部分。



In [0]:
# 冻结直到某一层（block5_onv1之前）的所有层
base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=50)